In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import os
from xgboost import XGBClassifier
from sklearn.metrics import f1_score, accuracy_score
import json
import numpy as np
from tqdm import tqdm
import xgboost as xgb
import shap

C:\Users\elyes\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Global Explanation

In [ ]:
train_texts = pd.read_parquet('data/x_train_vec.parquet')
valid_texts = pd.read_parquet('data/x_valid_vec.parquet')

In [ ]:
def build_shap_values(label, sample):
    with open(f'data/{label}/label_mapping.txt') as f:
        content = f.read().replace("'",'"')
        class_names = list(json.loads(content).keys())

    model = xgb.Booster()
    model.load_model(f"models/xgb/{label}.json")
    
    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(sample.astype(float))
    expected_value= explainer.expected_value
    return explainer, shap_values, expected_value, class_names

## local Explanations

In [ ]:
import shap
S_samples = valid_texts
S_explainer, S_shap_values, S_expected_value, S_class_names = build_shap_values("scope", PR_samples)
S_class_names

In [ ]:
shap.initjs()
shap.force_plot(S_expected_value, S_shap_values[0],feature_names=valid_texts.columns)